# Deep Learning e Object Recognition con dati da Google Street View

Recognizing photos taken from Google Street View linked to the cities of:
- <i><b>Manhattan</b></i>
- <i><b>Orlando</b></i>
- <i><b>Pittsburgh</b></i>

For this pourpose a CNN VGGG16 is taken into account. This CNN is pre-trained on a specific dataset called <i><b>Places</b></i>, and it has the task to train Deep Learning structures for Scene Context, Object Recognittion ecc.. The last part of the CNN is taken out to create a system of fully-connected layers <i>ad-hoc</i> for our scope.

In [1]:
from __future__ import division, print_function
import os

import warnings

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation, Dense, Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import GlobalMaxPooling2D
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.engine.topology import get_source_inputs
from keras.utils.data_utils import get_file
from keras.utils import layer_utils

Using TensorFlow backend.


In [3]:
WEIGHTS_PATH = 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/vgg16-hybrid1365_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/vgg16-hybrid1365_weights_tf_dim_ordering_tf_kernels_notop.h5'


def VGG16_Hybrid_3(include_top=True, weights='places', input_tensor=None, input_shape=None, pooling=None, classes=3):


    # Ripresa della migliore shape dell'input (ovvero 128 per il nsotro task)
    input_shape = _obtain_input_shape(input_shape, default_size=128, min_size=48, data_format=K.image_data_format(),
                                      require_flatten=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor


    # Primo blocco inerente ai primi due layer 
    x = Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same', kernel_regularizer=l2(0.0002),
                     activation='relu', name='block1_conv1')(img_input)

    x = Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same', kernel_regularizer=l2(0.0002),
                     activation='relu', name='block1_conv2')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block1_pool", padding='valid')(x)


    
    # Secondo blocco inerente ai layer 3 e 4. Cambia il numero di filtri
    x = Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same', kernel_regularizer=l2(0.0002),
                     activation='relu', name='block2_conv1')(x)

    x = Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same', kernel_regularizer=l2(0.0002),
                     activation='relu', name='block2_conv2')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block2_pool", padding='valid')(x)


    
    # Terzo blocco. Fa riferimento ai layer 5, 6 e 7. Anche qui cambia il numero di filtri utilizzati
    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same', kernel_regularizer=l2(0.0002),
                     activation='relu', name='block3_conv1')(x)

    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same', kernel_regularizer=l2(0.0002),
                     activation='relu', name='block3_conv2')(x)

    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same', kernel_regularizer=l2(0.0002),
                     activation='relu', name='block3_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block3_pool", padding='valid')(x)



    # Quarto blocco, facente rierimento ai layer 8, 9 e 10
    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same', kernel_regularizer=l2(0.0002),
                     activation='relu', name='block4_conv1')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same', kernel_regularizer=l2(0.0002),
                     activation='relu', name='block4_conv2')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same', kernel_regularizer=l2(0.0002),
                     activation='relu', name='block4_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block4_pool", padding='valid')(x)



    # Quinto ed ultimo blocco di default, inerente ai layer 11, 12 e 13
    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same', kernel_regularizer=l2(0.0002),
                     activation='relu', name='block5_conv1')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same', kernel_regularizer=l2(0.0002),
                     activation='relu', name='block5_conv2')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same', kernel_regularizer=l2(0.0002),
                     activation='relu', name='block5_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block5_pool", padding='valid')(x)


    # Parte di default dell'architettura base (mai utilizzata per il nostro scopo)
    if include_top:
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1_mod')(x)
        x = Dropout(0.5, name='drop_fc1_mod')(x)

        x = Dense(2048, activation='relu', name='fc2_mod')(x)
        x = Dropout(0.5, name='drop_fc2_mod')(x)

        x = Dense(3, activation='softmax', name='predictions_mod')(x)

    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Mi garantisce che il modello prenda in considerazione ogni potenziale predecessore di "input_tensor"
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input


    # Creazione del modello
    model = Model(inputs, x, name='vgg16-hybrid1365')

    # Procedure per caricare i pesi e per definire il back-end (si usa tf.Keras back-end)
    if weights == 'places':
        if include_top:
            weights_path = get_file('vgg16-hybrid1365_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models')
        else:
            weights_path = get_file('vgg16-hybrid1365_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models')

        model.load_weights(weights_path)

        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':
            if include_top:
                maxpool = model.get_layer(name='block5_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc1')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')


    elif weights is not None:
        model.load_weights(weights)



    return model

In [4]:
first_vgg16 = VGG16_Hybrid_3(weights='places', include_top=False, 
                      input_tensor=None, input_shape=(128,128,3),
                      pooling='avg',
                      classes=3)

In [4]:
first_vgg16.summary()

Model: "vgg16-hybrid1365"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)  

In [5]:
x = first_vgg16.output
x = tf.keras.layers.Dense(1024, activation='relu', name='fc1')(x)
x = tf.keras.layers.Dense(512, activation='relu', name='fc2')(x)
preds = tf.keras.layers.Dense(3, activation ='softmax', name='prediction')(x)

In [6]:
second_vgg16 = VGG16_Hybrid_3(weights='places', include_top=False, 
                      input_shape=(128,128,3),
                      pooling='avg',
                      classes=3)

In [7]:
second_vgg16 = tf.keras.models.Model(inputs=first_vgg16.input, outputs=preds)
second_vgg16.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [8]:
for i, layer in enumerate(second_vgg16.layers):
    print(i, layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 global_average_pooling2d
20 fc1
21 fc2
22 prediction


In [9]:
for layer in second_vgg16.layers[:19]:
    layer.trainable = False

for layer in second_vgg16.layers[19:]:
    layer.trainable = True
    
second_vgg16.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In questa maniera lasciamo inalterati i primi layer pre-addestrati sul dataset Places e addestriamo solamene gli ultimi layer posti "a mano"

In [10]:
import zipfile as zf
from PIL import Image

In [11]:
# Metodo per eliminare immagini di confusione

'''
import os
from os import listdir
from os.path import isfile, join
import shutil
import random


source_1 = r"C:\Users\david\Desktop\progetto machine learning\manhattan"
remove_1 = r"C:\Users\david\Desktop\da_rimuovere"

onlyfiles = [f for f in listdir(source_1) if isfile(join(source_1, f))]

da_eliminare = []

for image in onlyfiles:
    try:
        x = image.split('_')[1].split('.')[0]
        if x=='5': da_eliminare.append(image)
    except IndexError:
        continue

for image in da_eliminare:
    source_eliminare = source_1 + "\\" + image
    destination_eliminare = remove_1 + "\\" + image
    shutil.move(source_eliminare, destination_eliminare)
'''

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 112-113: truncated \UXXXXXXXX escape (<ipython-input-11-11a562c02b52>, line 29)

In [12]:
# Metodo per prendere in maniera random il testset dal dataset completo
'''


import os
from os import listdir
from os.path import isfile, join
import shutil
import random


source_1 = r"C:\Users\david\Desktop\progetto machine learning\manhattan"
destination_1 = r"C:\Users\david\Desktop\da_spostare"

onlyFiles = [f for f in listdir(source_1) if isfile(join(source_1, f))]
ciclo = len(onlyFiles) -1
remain = ciclo // 10
while remain>0:
    files_rem = [f for f in listdir(source_1) if isfile(join(source_1, f))]
    len_rem = len(files_rem) -1
    if len_rem==0: break
    value = random.randint(0,len_rem)
    image = files_rem[value]
    source_spostare = source_1 + "\\" + image
    destination_spostare = destination_1 + "\\" + image
    shutil.move(source_spostare, destination_spostare)
    files_rem.remove(image)
    remain = remain - 1
'''

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 114-115: truncated \UXXXXXXXX escape (<ipython-input-12-729c8b9d3f47>, line 29)

In [13]:
'''
files = zf.ZipFile("town_zip\manhattan.zip", 'r')
files.extractall('town_training')
files.close()
'''

'\nfiles = zf.ZipFile("town_zip\\manhattan.zip", \'r\')\nfiles.extractall(\'town_training\')\nfiles.close()\n'

In [14]:
'''
files = zf.ZipFile("town_zip\orlando.zip", 'r')
files.extractall('town_training')
files.close()
'''

'\nfiles = zf.ZipFile("town_zip\\orlando.zip", \'r\')\nfiles.extractall(\'town_training\')\nfiles.close()\n'

In [15]:
'''
files = zf.ZipFile("town_zip\pittsburgh.zip", 'r')
files.extractall('town_training')
files.close()
'''

'\nfiles = zf.ZipFile("town_zip\\pittsburgh.zip", \'r\')\nfiles.extractall(\'town_training\')\nfiles.close()\n'

### Test set

In [16]:
'''
files = zf.ZipFile("town_zip\manhattan_test.zip", 'r')
files.extractall('town_test')
files.close()
'''

'\nfiles = zf.ZipFile("town_zip\\manhattan_test.zip", \'r\')\nfiles.extractall(\'town_test\')\nfiles.close()\n'

In [17]:
'''
files = zf.ZipFile("town_zip\orlando_test.zip", 'r')
files.extractall('town_test')
files.close()
'''

'\nfiles = zf.ZipFile("town_zip\\orlando_test.zip", \'r\')\nfiles.extractall(\'town_test\')\nfiles.close()\n'

In [18]:
'''
files = zf.ZipFile("town_zip\pittsburgh_test.zip", 'r')
files.extractall('town_test')
files.close()
'''

'\nfiles = zf.ZipFile("town_zip\\pittsburgh_test.zip", \'r\')\nfiles.extractall(\'town_test\')\nfiles.close()\n'

In [19]:
# Metodo per ridimensionare le immagini e non aumentare troppo il carico computazionale
def resize(list_of_subdir):
    for folder in list_of_subdir:
        path = folder
        dirs = os.listdir(path)
        for item in dirs:
            if os.path.isfile(path+item):
                im = Image.open(path+item)
                f, e = os.path.splitext(path+item)
                image = im.resize((128,128), Image.ANTIALIAS)
                image.save(f + e, 'JPEG', quality=90)

In [20]:
### Resize della parte training

list_training = []
manhattan_training = 'town_training\manhattan\\'
orlando_training = 'town_training\orlando\\'
pittsburgh_training = 'town_training\\pittsburgh\\'

#list_training.append(manhattan_training)
#list_training.append(orlando_training)
list_training.append(pittsburgh_training)

In [21]:
# Metodo commentato perchè già effettuato (le immagini sono già resizate)

# resize(list_training)

In [22]:
### Resize della parte training

list_test = []
manhattan_test = 'town_test\manhattan\\'
orlando_test = 'town_test\orlando\\'
pittsburgh_test = 'town_test\\pittsburgh\\'

list_test.append(manhattan_test)
list_test.append(orlando_test)
list_test.append(pittsburgh_test)

list_test

['town_test\\manhattan\\', 'town_test\\orlando\\', 'town_test\\pittsburgh\\']

In [23]:
# Metodo commentato perchè già effettuato (le immagini sono già resizate)

# resize(list_test)

In [24]:
# Creazione del train_datagen e train_genrator che mi definiscon il dataset di training
train_town_dir = 'town_training'

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

batch_size = 32

train_generator = train_datagen.flow_from_directory(train_town_dir, target_size = (128, 128), color_mode = 'rgb',
                                                   batch_size = batch_size, classes=['manhattan', 'orlando', 'pittsburgh'],
                                                   class_mode = 'categorical', shuffle = True)

Found 11437 images belonging to 3 classes.


In [26]:
second_vgg16.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [39]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os

In [40]:
checkpoint_path = "principal_checkpoint\pc.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

In [41]:
second_vgg16.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [42]:
# Commentato anche se già effettuato 
# Non serve rifarlo per non adatare troppo il mdoello ai soli dati di training e rischiare l'overfitting

#history = second_vgg16.fit_generator(generator = train_generator, steps_per_epoch=train_generator.n//train_generator.batch_size, epochs = 2)

In [43]:
second_vgg16.save('principal_callback\principal_model.h5')

In [44]:
retrieval_second_vgg16 = tf.keras.models.load_model('principal_callback\principal_model.h5')

In [45]:
retrieval_second_vgg16.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [46]:
retrieval_second_vgg16.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [47]:
history = retrieval_second_vgg16.fit_generator(generator = train_generator, steps_per_epoch=train_generator.n//train_generator.batch_size, epochs = 1, callbacks=[cp_callback])

356/357 [============================>.] - ETA: 16s - loss: 0.7046 - accuracy: 0.8161
Epoch 00001: saving model to principal_checkpoint\pc.ckpt
357/357 [==============================] - 5742s 16s/step - loss: 0.7045 - accuracy: 0.8161


In [48]:
untrained_vgg16 = tf.keras.models.load_model('principal_callback\principal_model.h5')

In [49]:
untrained_vgg16.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [50]:
checkpoint_path

'principal_checkpoint\\pc.ckpt'

In [51]:
untrained_vgg16.load_weights(checkpoint_path)

In [52]:
for layer in untrained_vgg16.layers:
    prova_weights = layer.get_weights()

In [53]:
prova_weights

[array([[-0.03851825, -0.05813703, -0.05152307],
        [-0.03250661, -0.03040353,  0.01749098],
        [-0.0162875 , -0.0280205 , -0.05986626],
        ...,
        [ 0.0077372 ,  0.06112268,  0.0180974 ],
        [-0.08625552, -0.0074782 ,  0.04094208],
        [ 0.00051688, -0.02932363, -0.0071863 ]], dtype=float32),
 array([ 0.00107285, -0.00654078,  0.00546349], dtype=float32)]

In [54]:
for layer in retrieval_second_vgg16.layers:
    retrieval_weights = layer.get_weights()

In [55]:
retrieval_weights

[array([[-0.03851825, -0.05813703, -0.05152307],
        [-0.03250661, -0.03040353,  0.01749098],
        [-0.0162875 , -0.0280205 , -0.05986626],
        ...,
        [ 0.0077372 ,  0.06112268,  0.0180974 ],
        [-0.08625552, -0.0074782 ,  0.04094208],
        [ 0.00051688, -0.02932363, -0.0071863 ]], dtype=float32),
 array([ 0.00107285, -0.00654078,  0.00546349], dtype=float32)]

In [2]:
# Librerie da importare

import os
from os import listdir
from os.path import isfile, join
import shutil
import random

In [83]:
def create_ground_truth():
    manhattan_test = r"town_test\manhattan"
    orlando_test = r"town_test\orlando"
    pittsburgh_test = r"town_test\pittsburgh"

    manhattan_files = [f for f in listdir(manhattan_test) if isfile(join(manhattan_test, f))]
    orlando_files = [f for f in listdir(orlando_test) if isfile(join(orlando_test, f))]
    pittsburgh_files = [f for f in listdir(pittsburgh_test) if isfile(join(pittsburgh_test, f))]

    all_truth  = []
    for file in manhattan_files:
        man = (file, "manhattan")
        all_truth.append(man)
    
    for filo in orlando_files:
        orl = (filo, "orlando")
        all_truth.append(orl)
    
    for fila in pittsburgh_files:
        pitt = (fila, "pittsburgh")
        all_truth.append(pitt)
        
    return all_truth

In [88]:
def write_ground_truth(all_truth):
    string_truth = []
    for tupla in all_truth:
        val1 = str(tupla[0])
        val2 = str(tupla[1])
        val3 = val1 + " " + val2
        string_truth.append(val3)


    with open('ground_truth.txt', 'w') as f:
        for item in string_truth:
            f.write("%s\n" % item)

In [3]:
def read_txt(name):
    file1 = open(name, 'r') 
    Lines = file1.readlines() 

    reader = []
    count = 0 
    for line in Lines: 
        val1 = line.split(" ")[0]
        val2 = line.split(" ")[1][:-1]
        val3 = (val1, val2)
        reader.append(val3)
        
    return reader

In [94]:
all_truth = create_ground_truth()
write_ground_truth(all_truth)
ground_truth = read_txt('ground_truth.txt')

In [95]:
ground_truth

[('001101_1.jpg', 'manhattan'),
 ('001102_0.jpg', 'manhattan'),
 ('001104_4.jpg', 'manhattan'),
 ('001106_0.jpg', 'manhattan'),
 ('001107_1.jpg', 'manhattan'),
 ('001107_3.jpg', 'manhattan'),
 ('001109_0.jpg', 'manhattan'),
 ('001109_2.jpg', 'manhattan'),
 ('001110_4.jpg', 'manhattan'),
 ('001111_1.jpg', 'manhattan'),
 ('001112_0.jpg', 'manhattan'),
 ('001112_2.jpg', 'manhattan'),
 ('001112_3.jpg', 'manhattan'),
 ('001112_4.jpg', 'manhattan'),
 ('001114_1.jpg', 'manhattan'),
 ('001115_1.jpg', 'manhattan'),
 ('001115_4.jpg', 'manhattan'),
 ('001116_4.jpg', 'manhattan'),
 ('001117_4.jpg', 'manhattan'),
 ('001120_0.jpg', 'manhattan'),
 ('001122_0.jpg', 'manhattan'),
 ('001122_2.jpg', 'manhattan'),
 ('001124_2.jpg', 'manhattan'),
 ('001125_0.jpg', 'manhattan'),
 ('001125_3.jpg', 'manhattan'),
 ('001126_2.jpg', 'manhattan'),
 ('001130_2.jpg', 'manhattan'),
 ('001131_2.jpg', 'manhattan'),
 ('001133_3.jpg', 'manhattan'),
 ('001134_1.jpg', 'manhattan'),
 ('001135_1.jpg', 'manhattan'),
 ('00113

In [98]:
# Creazione del train_datagen e train_genrator che mi definiscon il dataset di training

test_town_dir = 'test'

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

test_generator = test_datagen.flow_from_directory(directory=test_town_dir, target_size=(128,128), color_mode="rgb",
                                                  batch_size=1, class_mode=None, shuffle=False, seed=42)

Found 2745 images belonging to 1 classes.


In [99]:
retrieval_vgg16 = tf.keras.models.load_model('principal_callback\principal_model.h5')
checkpoint_path = "principal_checkpoint\pc.ckpt"
retrieval_vgg16.load_weights(checkpoint_path)

In [101]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
prediction = retrieval_vgg16.predict_generator(test_generator,steps=STEP_SIZE_TEST,verbose=1)

2745/2745 [==============================] - 448s 163ms/step


In [110]:
# Import delle librerie necessarie
import numpy as np
import pandas as pd

In [112]:
# la variabile prediction contiene i valori della softmax inerenti alle 3 classi presenti nel modello
predicted_class_indices=np.argmax(prediction,axis=1)

In [113]:
# Definizione delle predizioni corrette
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [114]:
# Creazione DataFrame opportuno
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)

In [115]:
results

,Filename,Predictions
0,test_folder\001101_1.jpg,orlando
1,test_folder\001102_0.jpg,manhattan
2,test_folder\001104_4.jpg,manhattan
3,test_folder\001106_0.jpg,manhattan
4,test_folder\001107_1.jpg,manhattan
...,...,...
2740,test_folder\009402_4.jpg,pittsburgh
2741,test_folder\009403_2.jpg,pittsburgh
2742,test_folder\009405_4.jpg,pittsburgh
2743,test_folder\009409_1.jpg,pittsburgh


In [124]:
def read_results(results_dataframe):
    risultati_pred = []
    for index, row in results_dataframe.iterrows():
        image  = row['Filename'].split('\\')[1]
        pred_value = row['Predictions']
        tup = (image, pred_value)
        risultati_pred.append(tup)
    
    return risultati_pred

In [129]:
# all_truth
def write_prediction(preds):
    string_pred = []
    for tupla in preds:
        val1 = str(tupla[0])
        val2 = str(tupla[1])
        val3 = val1 + " " + val2
        string_pred.append(val3)


    with open('predictions.txt', 'w') as f:
        for item in string_pred:
            f.write("%s\n" % item)

In [130]:
preds = read_results(results)
write_prediction(preds)

In [4]:
ground_truth = read_txt('ground_truth.txt')
predict = read_txt('predictions.txt')

In [5]:
ground_truth

[('001101_1.jpg', 'manhattan'),
 ('001102_0.jpg', 'manhattan'),
 ('001104_4.jpg', 'manhattan'),
 ('001106_0.jpg', 'manhattan'),
 ('001107_1.jpg', 'manhattan'),
 ('001107_3.jpg', 'manhattan'),
 ('001109_0.jpg', 'manhattan'),
 ('001109_2.jpg', 'manhattan'),
 ('001110_4.jpg', 'manhattan'),
 ('001111_1.jpg', 'manhattan'),
 ('001112_0.jpg', 'manhattan'),
 ('001112_2.jpg', 'manhattan'),
 ('001112_3.jpg', 'manhattan'),
 ('001112_4.jpg', 'manhattan'),
 ('001114_1.jpg', 'manhattan'),
 ('001115_1.jpg', 'manhattan'),
 ('001115_4.jpg', 'manhattan'),
 ('001116_4.jpg', 'manhattan'),
 ('001117_4.jpg', 'manhattan'),
 ('001120_0.jpg', 'manhattan'),
 ('001122_0.jpg', 'manhattan'),
 ('001122_2.jpg', 'manhattan'),
 ('001124_2.jpg', 'manhattan'),
 ('001125_0.jpg', 'manhattan'),
 ('001125_3.jpg', 'manhattan'),
 ('001126_2.jpg', 'manhattan'),
 ('001130_2.jpg', 'manhattan'),
 ('001131_2.jpg', 'manhattan'),
 ('001133_3.jpg', 'manhattan'),
 ('001134_1.jpg', 'manhattan'),
 ('001135_1.jpg', 'manhattan'),
 ('00113

In [6]:
predict

[('001101_1.jpg', 'orlando'),
 ('001102_0.jpg', 'manhattan'),
 ('001104_4.jpg', 'manhattan'),
 ('001106_0.jpg', 'manhattan'),
 ('001107_1.jpg', 'manhattan'),
 ('001107_3.jpg', 'pittsburgh'),
 ('001109_0.jpg', 'manhattan'),
 ('001109_2.jpg', 'manhattan'),
 ('001110_2.jpg', 'manhattan'),
 ('001110_4.jpg', 'manhattan'),
 ('001111_1.jpg', 'manhattan'),
 ('001111_3.jpg', 'manhattan'),
 ('001112_0.jpg', 'manhattan'),
 ('001112_2.jpg', 'manhattan'),
 ('001112_3.jpg', 'manhattan'),
 ('001112_4.jpg', 'manhattan'),
 ('001114_1.jpg', 'manhattan'),
 ('001115_1.jpg', 'pittsburgh'),
 ('001115_4.jpg', 'manhattan'),
 ('001116_4.jpg', 'manhattan'),
 ('001117_4.jpg', 'manhattan'),
 ('001118_1.jpg', 'manhattan'),
 ('001120_0.jpg', 'manhattan'),
 ('001122_0.jpg', 'manhattan'),
 ('001122_2.jpg', 'manhattan'),
 ('001124_2.jpg', 'manhattan'),
 ('001125_0.jpg', 'manhattan'),
 ('001125_3.jpg', 'manhattan'),
 ('001126_2.jpg', 'manhattan'),
 ('001130_2.jpg', 'manhattan'),
 ('001130_3.jpg', 'manhattan'),
 ('00113

In [7]:
# ground_truth
# predict
def evaluate_mymodel(ground_truth, predict):
    total_count = 0
    right_count = 0
    for image_pred, pred_pred in predict:
        total_count += 1
        for image_ground, pred_ground in ground_truth:
            if image_pred == image_ground:
                if pred_pred == pred_ground:
                    right_count += 1
                else: continue
            else: continue
    return right_count/total_count

In [8]:
pred_value = evaluate_mymodel(ground_truth, predict)

In [10]:
pred_value

0.7380692167577414

In [11]:
pred_value = pred_value * 100
print("La predizione ha ripostato un'accuracy del " + str(round(pred_value, 3)) + "%")

La predizione ha ripostato un'accuracy del 73.807%
